In [ ]:
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# setup
import torch
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
import pandas as pd
import io

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving merged_annotated_comments.xlsx to merged_annotated_comments.xlsx


In [ ]:
data = pd.read_excel('merged_annotated_comments.xlsx')

In [ ]:
data.head()

,comment_user,comment_time,reply_count,post_comment_count,Channel,comment,title,comment_id,language,predicted_label,label,Unnamed: 11
0,@greencrow72,NaN,0,650,Trump,posterity the most important word in the pream...,Hall of Fame Quarterback Brett Favre Speaks at...,63344,en,negative,normal,NaN
1,@RyanSmith-sr9il,NaN,0,"2,764",Harris,why was this even recorded propaganda,Barack and Michelle Obama Endorse Kamala Harri...,102690,en,negative,normal,NaN
2,@trentonspeakstv,NaN,1,"1,051",Harris,that was powerful,Vice President Kamala Harris On Democracy & Th...,77006,en,negative,normal,NaN
3,@-kitn-kittilsvogel1958,NaN,0,"4,807",Trump,axios itself referred to harris as bidens bord...,Meet San Francisco Liberal Kamala Harris,8737,en,negative,normal,NaN
4,@mariuszmoraw3571,NaN,0,"9,999",Trump,the fact not even democrats choose her or vote...,Meet the Real Kamala Harris.,78274,en,negative,normal,NaN


In [ ]:
data = data[['comment', 'label']]
data = data.rename(columns={'comment': 'text'})

In [ ]:
type(data['text'][0])

str

In [ ]:
data.head()

,text,label
0,posterity the most important word in the pream...,normal
1,why was this even recorded propaganda,normal
2,that was powerful,normal
3,axios itself referred to harris as bidens bord...,normal
4,the fact not even democrats choose her or vote...,normal


In [ ]:
label_mapping = {'normal': 0, 'insult': 1, }
data['label'] = data['label'].map(label_mapping)

# Check for missing or invalid mappings
if data['label'].isnull().any():
    print("Found missing or invalid labels. Rows with invalid labels will be removed.")
    # Drop rows with invalid labels
    data = data.dropna(subset=['label'])

# Convert to integers
data['label'] = data['label'].astype(int)

# Verify data types
print(data['label'].dtype)

Found missing or invalid labels. Rows with invalid labels will be removed.
int64


In [ ]:
data.head()

,text,label
0,posterity the most important word in the pream...,0
1,why was this even recorded propaganda,0
2,that was powerful,0
3,axios itself referred to harris as bidens bord...,0
4,the fact not even democrats choose her or vote...,0


In [ ]:
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    data['text'], data['label'], test_size=0.2, random_state=42
)

In [ ]:
val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42  # Split the temp set equally
)

In [ ]:
train_dataset = Dataset.from_dict({'text': train_texts, 'label': train_labels})
val_dataset = Dataset.from_dict({'text': val_texts, 'label': val_labels})
dataset = DatasetDict({'train': train_dataset, 'validation': val_dataset})

In [ ]:
print(type(train_texts[0]), train_texts[0])

<class 'str'> posterity the most important word in the preamble of the united states constitution


load tokenizer and pre-trained model

In [ ]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['text'])  # Keep only tokenized inputs and labels


Map:   0%|          | 0/1581 [00:00<?, ? examples/s]

Map:   0%|          | 0/198 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3 classes
model.to(device)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

Define Training arguments

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=50,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,  # Adjust based on available time
    learning_rate=2e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none",  # Disable reporting to W&B or similar services
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.0 MB/s eta 0:00:00


In [ ]:
from evaluate import load
accuracy = load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return accuracy.compute(predictions=predictions, references=labels)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-24-8dc601782a9a>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Fine tuning of the model

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.451600,0.345148,0.883838
2,0.217300,0.308350,0.888889
3,0.128400,0.349208,0.893939


TrainOutput(global_step=297, training_loss=0.2305558378046209, metrics={'train_runtime': 481.9617, 'train_samples_per_second': 9.841, 'train_steps_per_second': 0.616, 'total_flos': 1247946940283904.0, 'train_loss': 0.2305558378046209, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./fine_tuned_model_hate_speech")
tokenizer.save_pretrained("./fine_tuned_model_hate_speech")

print("Fine-tuning complete. Model saved!")

Fine-tuning complete. Model saved!


In [ ]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
encoded_test = tokenizer(test_texts.tolist(), padding=True, truncation=True, max_length=512, return_tensors="pt")

In [ ]:
input_ids = encoded_test['input_ids'].to(device)


In [ ]:
test_labels.head()

,label
1616,0
297,0
693,0
237,0
71,0


In [ ]:
test_labels = torch.tensor(test_labels.tolist()).to(device)

In [ ]:

attention_mask = encoded_test['attention_mask'].to(device)


In [ ]:
model.eval()
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    predictions = torch.argmax(outputs.logits, dim=-1)

In [ ]:
y_pred = predictions.cpu().numpy()
y_true = test_labels.cpu().numpy()

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
print("Accuracy:", accuracy_score(y_true, y_pred))
print("\nClassification Report:")
print(classification_report(y_true, y_pred))

Accuracy: 0.9090909090909091

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.97      0.95       181
           1       0.45      0.29      0.36        17

    accuracy                           0.91       198
   macro avg       0.70      0.63      0.65       198
weighted avg       0.89      0.91      0.90       198



apply classifier to all comments


In [ ]:
uploaded = files.upload()


Saving comments_only.csv to comments_only.csv


In [ ]:
new_data = pd.read_csv('comments_only.csv')

In [ ]:
new_data.head()

,comment_user,comment_time,reply_count,post_comment_count,Channel,comment,title,comment_id,language
0,@JConlon96,2024-11-07 09:54:00,0.0,486,Harris,harris,Big Things | Harris-Walz 2024,25828.0,en
1,@BookMama72,2024-11-07 14:54:00,11.0,480,Harris,im nauseously optimistic this morning lets do ...,Girl Dad | Harris-Walz 2024,41128.0,en
2,@ColeFNCS,2024-11-07 21:54:00,1.0,480,Harris,just voted ballot red lets go america trump i...,Girl Dad | Harris-Walz 2024,46144.0,en
3,@bryanpotter3882,2024-11-07 20:54:00,0.0,413,Harris,kamala harris for president,Good Sense | Harris-Walz 2024,46926.0,no
4,@rhogenelabata2774,2024-11-07 16:54:00,0.0,413,Harris,kentuckian republican here for harriswalz,Good Sense | Harris-Walz 2024,48831.0,en


In [ ]:
comments = new_data['comment'].astype(str).tolist()
encodings = tokenizer(
    comments,
    truncation=True,
    padding=True,
    max_length=512,
    return_tensors="pt"
)

In [ ]:
batch_size = 32  # Number of comments per batch
device = "cuda" if torch.cuda.is_available() else "cpu"


In [ ]:
def predict_in_batches(comments, batch_size=32):
    """
    Predict labels for comments in batches to reduce memory usage.
    """
    predictions = []
    model.eval()

    with torch.no_grad():
        for i in range(0, len(comments), batch_size):
            # Get the current batch
            batch = comments[i:i + batch_size]

            # Tokenize the batch
            encodings = tokenizer(
                batch,
                truncation=True,
                padding=True,
                max_length=512,
                return_tensors="pt"
            )

            # Move inputs to the appropriate device
            input_ids = encodings['input_ids'].to(device)
            attention_mask = encodings['attention_mask'].to(device)

            # Get model outputs
            outputs = model(input_ids, attention_mask=attention_mask)
            batch_predictions = torch.argmax(outputs.logits, dim=-1)

            # Convert predictions to CPU and append
            predictions.extend(batch_predictions.cpu().numpy())

    return predictions

In [ ]:
comments = new_data['comment'].astype(str).tolist()
predicted_labels = predict_in_batches(comments, batch_size=batch_size)

In [ ]:
label_mapping = {0: 'normal critic', 1: 'insult'}
predicted_labels = [label_mapping[pred] for pred in predicted_labels]


In [ ]:
new_data['Predicted_Label'] = predicted_labels

In [ ]:
output_file = 'comments_with_predictions.xlsx'
new_data.to_excel(output_file, index=False)
print(f"Predictions saved to {output_file}")

Predictions saved to comments_with_predictions.xlsx


In [ ]:
from google.colab import files

files.download('comments_with_predictions.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
new_data.head()

,comment_user,comment_time,reply_count,post_comment_count,Channel,comment,title,comment_id,language,Predicted_Label
0,@JConlon96,2024-11-07 09:54:00,0.0,486,Harris,harris,Big Things | Harris-Walz 2024,25828.0,en,normal critic
1,@BookMama72,2024-11-07 14:54:00,11.0,480,Harris,im nauseously optimistic this morning lets do ...,Girl Dad | Harris-Walz 2024,41128.0,en,normal critic
2,@ColeFNCS,2024-11-07 21:54:00,1.0,480,Harris,just voted ballot red lets go america trump i...,Girl Dad | Harris-Walz 2024,46144.0,en,normal critic
3,@bryanpotter3882,2024-11-07 20:54:00,0.0,413,Harris,kamala harris for president,Good Sense | Harris-Walz 2024,46926.0,no,normal critic
4,@rhogenelabata2774,2024-11-07 16:54:00,0.0,413,Harris,kentuckian republican here for harriswalz,Good Sense | Harris-Walz 2024,48831.0,en,normal critic


In [ ]:
insult_comments = new_data[new_data['Predicted_Label'] == 'insult']


In [ ]:
output_file = 'hate_speech_comments.xlsx'
insult_comments.to_excel(output_file, index=False)
print(f"Predictions saved to {output_file}")

Predictions saved to hate_speech_comments.xlsx


In [ ]:
from google.colab import files

files.download('hate_speech_comments.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
random_samples = insult_comments.sample(n=5)
sampled_table = random_samples[['Channel', 'comment', 'Predicted_Label']]
display(sampled_table)


,Channel,comment,Predicted_Label
52151,Trump,kamala had her white voice on tonight,insult
49763,Trump,she is rude and condescending,insult
39717,Harris,kamella is just maduro in a pantsuit,insult
10495,Harris,this woman is clearly deranged,insult
6189,Harris,broo why is kamala so cringe she is so fake,insult


In [ ]:
total_insults = len(insult_comments)
channel_counts = insult_comments['Channel'].value_counts()
channel_percentages = (channel_counts / total_insults) * 100
print("Percentage of insult comments by channel:")
print(channel_percentages)


Percentage of insult comments by channel:
Channel
Trump     60.985031
Harris    39.014969
Name: count, dtype: float64
